# Deep MNIST for Experts

## Load MNIST Data

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
#this is the num of neurals of the full-connected layer, 
#we can change to any number you want
NUM_FC = 1024
##max training steps
MAX_Step = 1000

## Start Tensorflow InteractiveSession

In [5]:
sess = tf.InteractiveSession()

## Build a softmax regression model

In [6]:
# shape = [None, 784], 'None' means that the first dim can be any size, corresponding to the batch size
# 784 corresponding the single flattened img. 10 corresponding to the 10 digits
x= tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [7]:
# W :784 input features and 10 outputs
# b : 10 classes
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [8]:
sess.run(tf.initialize_all_variables())

## Predict Class and Cost Function

In [9]:
#compute the prob that assigned to each class
y = tf.nn.softmax(tf.matmul(x, W) + b)
#cost function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y), reduction_indices=[1]))

## Train the Model

In [10]:
#step length = 0.5
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [11]:
for i in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

## Evaluate the Model

In [12]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_:mnist.test.labels}))

0.9114


## Build a Multiplayer Convolutional Network

### Weight Initialization

In [13]:
#initializa weights with small noise for symmetry breaking and to prevent 0 gradients
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape): 
    # remember to add "shape = " ,or  you will get error later
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### Convolution and Pooling

In [14]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], 
                          strides=[1, 2, 2, 1], padding='SAME')

### First Convolutional Layer

In [15]:
#[filter_height, filter_width, in_channels, out_channels]
# compute 32 features
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [16]:
#reshape the x to 4d tensor [batch, in_height, in_width, in_channels]
# [-1, 28, 28, 1]: "-1" means to adjust the dim 
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Second Convolutional Layer

In [17]:
#compute 64 features
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

In [18]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Densely Connected Layer

In [19]:

W_fc1 = weight_variable([7 * 7 * 64, NUM_FC])
b_fc1 = bias_variable([NUM_FC])

In [20]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Dropout

In [21]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout Layer

In [22]:
# add a softmax layer
W_fc2 = weight_variable([NUM_FC, 10])
b_fc2 = weight_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### Train and Evaluate the Model

In [23]:
batch = mnist.train.next_batch(50)
print batch[0].shape

(50, 784)


In [24]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())

for i in range(MAX_Step):
    batch = mnist.train.next_batch(50)
    if i%20 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob :1})
        print("step %d, train_accuracy: %g" %(i, train_accuracy))
    train_step.run(feed_dict={x:batch[0], y_: batch[1], keep_prob: 0.5})
    #print sess.run(accuracy, feed_dict={x:batch[0], y_: batch[1], keep_prob: 0.5})
print("test_accuracy: %g" %(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0})))

step 0, train_accuracy: 0.04
step 20, train_accuracy: 0.44
step 40, train_accuracy: 0.64
step 60, train_accuracy: 0.84
step 80, train_accuracy: 0.7
step 100, train_accuracy: 0.88
step 120, train_accuracy: 0.9
step 140, train_accuracy: 0.88
step 160, train_accuracy: 1
step 180, train_accuracy: 0.86
step 200, train_accuracy: 0.9
step 220, train_accuracy: 0.94
step 240, train_accuracy: 0.94
step 260, train_accuracy: 0.9
step 280, train_accuracy: 0.86
step 300, train_accuracy: 0.92
step 320, train_accuracy: 0.9
step 340, train_accuracy: 0.92
step 360, train_accuracy: 0.96
step 380, train_accuracy: 0.94
step 400, train_accuracy: 0.92
step 420, train_accuracy: 0.88
step 440, train_accuracy: 0.92
step 460, train_accuracy: 0.94
step 480, train_accuracy: 1
step 500, train_accuracy: 0.88
step 520, train_accuracy: 0.96
step 540, train_accuracy: 0.94
step 560, train_accuracy: 0.96
step 580, train_accuracy: 0.98
step 600, train_accuracy: 0.98
step 620, train_accuracy: 0.98
step 640, train_accuracy: